In [1]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports
import dash_leaflet as dl
from dash import dcc
from dash import html
import plotly.express as pxz

from dash import dash_table
from dash.dependencies import Input, Output


# Configure the plotting routines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt


#### FIX ME ##### --DONE
# change animal_shelter and AnimalShelter to match your CRUD Python module file name and class name
from crud import AnimalShelter

# import for donut chart
import plotly.express as px

###########################
# Data Manipulation / Model
###########################
# FIX ME update with your username and password and CRUD Python module name. NOTE: You will
# likely need more variables for your constructor to handle the hostname and port of the MongoDB
# server, and the database and collection names --DONE
    
username = "aacuser"
password = "SNHU1234"
hostname = 'nv-desktop-services.apporto.com'       # added
port     = 31319                                   # added
DB       = 'AAC'  #'aac' -is all caps in show dbs  # added
COL      = 'animals'
shelter  = AnimalShelter(username, password, hostname, port, DB, COL)

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(shelter.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
print(df.columns)


Index(['rec_num', 'age_upon_outcome', 'animal_id', 'animal_type', 'breed',
       'color', 'date_of_birth', 'datetime', 'monthyear', 'name',
       'outcome_subtype', 'outcome_type', 'sex_upon_outcome', 'location_lat',
       'location_long', 'age_upon_outcome_in_weeks'],
      dtype='object')


In [2]:

#########################
# Dashboard Layout / View
#########################
app = JupyterDash('SimpleExample')


app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),

    # Header with logo and title
    html.Div([
        html.Img(src='/assets/Grazioso Salvare Logo.png', 
                 style={'height': '100px', 'margin-right': '15px'}), 
        html.H1('SNHU CS-340 Project Two: Rescue Animal Dashboard (Audrey Weaver)', 
                style={'display': 'inline-block', 'verticalAlign': 'middle', 'white-space': 'nowrap'})
    ], style={'display': 'flex', 'alignItems': 'center'}), 

    html.Hr(),
    
    
#     html.Center(html.B(html.H1(''))), --original title (no logo)


    #radio buttons for the preset filters
    html.Div([
        html.Label("Filter Options:", style={'font-weight': 'bold', 'margin-right': '12px'}),
        dcc.RadioItems(
            id='radio-filter',
            options=[
                # the four required filters
                {'label': 'Water Rescue', 'value': 'water_rescue'},
                {'label': 'Mountain Rescue', 'value': 'mountain_rescue'},
                {'label': 'Disaster Rescue', 'value': 'disaster_rescue'},
                {'label': 'Reset Filters', 'value': 'reset'}
            ],
            value='reset',  # resets to show all
            labelStyle={'display': 'inline-block', 'margin-right': '10px'}
        )
    ], style={'textAlign': 'left', 'margin-bottom': '10px'}),
    
    html.Hr(),
    
    # data table, map and other chart inside div container
    html.Div([
        dash_table.DataTable(
            id='datatable-id',
            columns=[{"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns],
            data=df.to_dict('records'),

            # set table functionality
            filter_action="native",
            sort_action="native",
            row_selectable="single",
            selected_rows=[0],
            page_size=10,

            # Table styling
            style_header={'backgroundColor': 'rgb(050,080,150)', 'fontWeight': 'bold', 'color': 'white'},
            style_cell={'textAlign': 'left', 'padding': '10px'},
        ),
        
        html.Br(),
        html.Hr(),  
        html.Div([
            # Donut Chart on left
            html.Div([
                dcc.Graph(id='donut-chart')
            ], style={'width': '48%', 'display': 'inline-block', 'padding': '10px', 'textAlign': 'center'}),

            # Map on right
            html.Div([
                html.Div(id='map-id', className='map-container')
            ], style={'width': '48%', 'display': 'inline-block', 'padding': '10px', 'textAlign': 'center'})
        ]
            , style={'display': 'flex', 'justify-content': 'center'}
#             , style={'justify-content': 'center'}
        )


]), ])


#############################################
# Interaction Between Components / Controller
#############################################
#This callback will highlight a row on the data table when the user selects it
# @app.callback(
#     Output('datatable-id', 'style_data_conditional'),
#     [Input('datatable-id', 'selected_columns')]
# )

def update_styles(selected_columns):
    #trying to avoid the none type error in styles
    if selected_columns is None:
        return []
    
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


In [3]:
# adding chart
@app.callback(
    [Output('datatable-id', 'data'),
     Output('donut-chart', 'figure')],
    [Input('radio-filter', 'value')]
)

# function to filter table and include donut chart
def update_data(filter_value):
    
    # load dataset (reset)
    filtered_df = df.copy()

    # Apply filters based on radio buttons
    if filter_value == 'water_rescue':
        filtered_df = filtered_df[
            (filtered_df['breed'].isin(['Labrador Retriever Mix',
                                        'Chesapeake Bay Retriever',
                                        'Newfoundland'])) &
            (filtered_df['sex_upon_outcome'] == 'Intact Female') &
            (filtered_df['age_upon_outcome_in_weeks'].between(26, 156)) 
            # show only available? omitting for consistency - see mountain_rescue 
#             (filtered_df['outcome_type'] != 'Euthanasia') &
#             (filtered_df['outcome_type'] != 'Return to Owner')
        ]
    
    elif filter_value == 'mountain_rescue':
        filtered_df = filtered_df[
            (filtered_df['breed'].isin(['German Shepherd',
                                        'Alaskan Malamute',
                                        'Old English Sheepdog',
                                        'Siberian Husky',
                                        'Rottweiler'])) &
            (filtered_df['sex_upon_outcome'] == 'Intact Male') &
            (filtered_df['age_upon_outcome_in_weeks'].between(26, 156)) 
            # show only available? - apparently only the below are available for mountain rescue :(
#             (filtered_df['outcome_type'] != 'Euthanasia') &
#             (filtered_df['outcome_type'] != 'Return to Owner')
        ]

    elif filter_value == 'disaster_rescue':
        filtered_df = filtered_df[
            (filtered_df['breed'].isin(['Doberman Pinscher',
                                        'German Shepherd',
                                        'Golden Retriever', 
                                        'Bloodhound', 
                                        'Rottweiler'])) &
            (filtered_df['age_upon_outcome_in_weeks'].between(20, 300))
            # show only available? omitting for consistency - see mountain_rescue 
#             (filtered_df['outcome_type'] != 'Euthanasia') &
#             (filtered_df['outcome_type'] != 'Return to Owner')
        ]

    # Reset to original dataset
    elif filter_value == 'reset':
        filtered_df = df.copy()

        
    # Donut Chart
    # get count of animals by breed
    breed_counts = filtered_df['breed'].value_counts().reset_index()
    breed_counts.columns = ['breed', 'count']

    # control number of breed labels showing if there are too many, > 10
    num_breeds = filtered_df['breed'].nunique()
    
    # Create the donut chart
    fig = px.pie(
        breed_counts,
        names='breed',
        values='count',
        hole=0.5,
        title="Breed Distribution"
    )

    # Add KPI in Center of visual for total number of pets
    fig.update_layout(
        annotations=[{
            "text": filtered_df.shape[0],
            "x": 0.5, "y": 0.5, "font_size": 20, "showarrow": False
        }]
    )
    
    fig.update_traces(textinfo='none' if num_breeds > 10 else 'percent')

    return filtered_df.to_dict('records'), fig 


In [4]:
# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])

def update_map(viewData, index):
#FIXME Add in the code for your geolocation chart

    if viewData is None or len(viewData) == 0:
        return dl.Map(center=[30.75, -97.48], zoom=10) #defaults to Austin, TX
    
    # use panadas to get dict data
    data = pd.DataFrame.from_dict(viewData)
    
    # set default selection to avoid errors
    row = 0 if index is None or len(index) == 0 else index[0]
    
    # handle misc errors
    try:
        lat   = float(data.iloc[row,13])
        lon   = float(data.iloc[row,14])
        breed = (data.iloc[row, 4])
        name  = (data.iloc[row, 9])
    except (IndexError, ValueError, KeyError):
        return dl.Map(center=[30.75, -97.48], zoom=5)
    
    #handle blank names
    if len(name) == 0:
        name = 'Ready to be named!'
    else:
        name
        
    
    # show map
    return dl.Map(
        style={'width': '1000px', 'height': '500px'},
        center=[lat, lon], zoom=8,
        children=[
            dl.TileLayer(id="base-layer-id"),
            dl.Marker(position=[lat, lon], children=[
                dl.Tooltip(breed),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(name)
                ])
            ])
        ]
    )              

app.run_server(debug=True)

Dash app running on http://127.0.0.1:13718/


In [5]:
#  get all of hte current versions required for readme
import jupyter_dash
import dash_leaflet
import dash
import plotly
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

print("JupyterDash:", jupyter_dash.__version__)
print("Dash Leaflet:", dash_leaflet.__version__)
print("Dash:", dash.__version__)
print("Plotly:", plotly.__version__)
print("NumPy:", np.__version__)
print("Pandas:", pd.__version__)
print("Matplotlib:", matplotlib.__version__)

JupyterDash: 0.4.2
Dash Leaflet: 0.1.23
Dash: 2.8.1
Plotly: 5.6.0
NumPy: 1.21.5
Pandas: 1.4.2
Matplotlib: 3.5.1
